# 프로젝트 개요
[출처(https://www.youtube.com/watch?app=desktop&v=7yAw1nPareM)](https://www.youtube.com/watch?app=desktop&v=7yAw1nPareM)

### 프로젝트 목표
- 미국 수퍼마켓의 매출 데이터를 분석하여 매출을 증대시키기 위한 전략을 수립

### 소스
- [데이터](https://community.tableau.com/s/question/0D54T00000CWeX8SAL/sample-superstore-sales-excelxls)
- [소스코드](https://github.com/AbhisheakSaraswat/PythonStreamlit/blob/main/Dashboard.py)

# 데이터 전처리

#### 데이터 준비

In [1]:
import streamlit as st
import plotly.express as px
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 데이터 불러오기
sales = pd.read_csv("./data/Superstore.csv")
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9994 non-null   int64  
 1   Order ID       9994 non-null   object 
 2   Order Date     9994 non-null   object 
 3   Ship Date      9994 non-null   object 
 4   Ship Mode      9994 non-null   object 
 5   Customer ID    9994 non-null   object 
 6   Customer Name  9994 non-null   object 
 7   Segment        9994 non-null   object 
 8   Country        9994 non-null   object 
 9   City           9994 non-null   object 
 10  State          9994 non-null   object 
 11  Postal Code    9994 non-null   int64  
 12  Region         9994 non-null   object 
 13  Product ID     9994 non-null   object 
 14  Category       9994 non-null   object 
 15  Sub-Category   9994 non-null   object 
 16  Product Name   9994 non-null   object 
 17  Sales          9994 non-null   float64
 18  Quantity

In [3]:
sales.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [4]:
# Order Date를 날짜형으로 변환
sales["Order Date"] = pd.to_datetime(sales["Order Date"])

# 기간 설정
start = pd.to_datetime("2017-01-01")
end = pd.to_datetime("2017-01-31")

# "Order Date" 컬럼을 기준으로 start와 end 사이의 
# ["Order ID", "Order Date", "Region, "State", "City"]데이터만 추출
df = sales[(sales["Order Date"] >= start) & (sales["Order Date"] <= end)][['Order ID', 'Order Date', 'Region', 'State', 'City']].copy()
df

,Order ID,Order Date,Region,State,City
439,CA-2017-157252,2017-01-20,East,New York,New York City
515,CA-2017-127432,2017-01-22,West,Montana,Great Falls
516,CA-2017-127432,2017-01-22,West,Montana,Great Falls
517,CA-2017-127432,2017-01-22,West,Montana,Great Falls
518,CA-2017-127432,2017-01-22,West,Montana,Great Falls
...,...,...,...,...,...
9865,CA-2017-113278,2017-01-14,Central,Indiana,Richmond
9914,CA-2017-160927,2017-01-29,Central,Iowa,Marion
9915,CA-2017-160927,2017-01-29,Central,Iowa,Marion
9916,CA-2017-160927,2017-01-29,Central,Iowa,Marion


In [5]:
# 초기값 설정
region = ["West", "Central"]
state = ["Indiana"]
city = None

if region is not None:
    df_region = df[df["Region"].isin(region)].copy()
else:
    df_region = df.copy()

if state is not None:
    df_state = df[df["State"].isin(state)].copy()
else:
    df_state = df.copy()

if city is not None:
    df_city = df[df["City"].isin(city)].copy()
else:
    df_city = df.copy()

# region, state, city에 따른 데이터프레임 출력
df_filtered = pd.merge(df_region, df_state, how="inner")
df_filtered = pd.merge(df_filtered, df_city, how="inner")

df_filtered

,Order ID,Order Date,Region,State,City
0,CA-2017-113278,2017-01-14,Central,Indiana,Richmond
1,CA-2017-113278,2017-01-14,Central,Indiana,Richmond
2,CA-2017-113278,2017-01-14,Central,Indiana,Richmond
3,CA-2017-113278,2017-01-14,Central,Indiana,Richmond
4,CA-2017-113278,2017-01-14,Central,Indiana,Richmond
...,...,...,...,...,...
507,CA-2017-113278,2017-01-14,Central,Indiana,Richmond
508,CA-2017-113278,2017-01-14,Central,Indiana,Richmond
509,CA-2017-113278,2017-01-14,Central,Indiana,Richmond
510,CA-2017-113278,2017-01-14,Central,Indiana,Richmond
